In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
data_dir = "/home/pleroy/DATA/2017_11_ietr/"
fname = data_dir + "ramp 600us.csv"
t, v1, v2 = np.genfromtxt( fname, delimiter=',', skip_header=2, unpack=True )

In [3]:
plt.figure()
plt.plot( t, v1, label="v1" )
plt.plot( t, v2, label="v2" )
plt.grid()
plt.legend()

In [4]:
binFilename = data_dir + "scope_2.bin"
#binFilename = data_dir + "myFileName.bin"

In [5]:
def importAgilentBin(inputFilename, waveform):
    
    with open(binFilename, 'r') as fileId:

        # read file header
        fileCookie = np.fromfile(fileId, 'int8', 2) # 65 71 => 'AG'
        fileVersion = np.fromfile(fileId, 'int8', 2)
        fileSize = np.fromfile(fileId, 'int32', 1)
        nWaveforms = np.fromfile(fileId, 'int32', 1)[0]

        print("fileCookie = {}, fileVersion = {}, fileSize = {}, nWaveforms = {}".format( 
        fileCookie, fileVersion, fileSize, nWaveforms )
         )

        # determine which waveform to read
        waveformSelect = waveform

        for waveformIndex in range( nWaveforms ):
            # read waveform header
            headerSize = np.fromfile(fileId, 'int32', 1)[0]
            bytesLeft = headerSize - 4
    
            waveformType = np.fromfile(fileId, 'int32', 1)
            bytesLeft = bytesLeft - 4
    
            nWaveformBuffers = np.fromfile(fileId, 'int32', 1)[0]
            bytesLeft = bytesLeft - 4
    
            nPoints = np.fromfile(fileId, 'int32', 1)[0]
            bytesLeft = bytesLeft - 4
    
            count = np.fromfile(fileId, 'int32', 1)
            bytesLeft = bytesLeft - 4
    
            xDisplayRange = np.fromfile(fileId, 'float32', 1)
            bytesLeft = bytesLeft - 4
    
            xDisplayOrigin = np.fromfile(fileId, 'float64', 1)
            bytesLeft = bytesLeft - 8
    
            xIncrement = np.fromfile(fileId, 'float64', 1)
            bytesLeft = bytesLeft - 8
    
            xOrigin = np.fromfile(fileId, 'float64', 1)
            bytesLeft = bytesLeft - 8
    
            xUnits = np.fromfile(fileId, 'int32', 1)
            bytesLeft = bytesLeft - 4
    
            yUnits = np.fromfile(fileId, 'int32', 1)
            bytesLeft = bytesLeft - 4
    
            dateString = np.fromfile(fileId, 'int8', 16)
            bytesLeft = bytesLeft - 16
    
            timeString = np.fromfile(fileId, 'int8', 16)
            bytesLeft = bytesLeft - 16
    
            frameString = np.fromfile(fileId, 'int8', 24)
            bytesLeft = bytesLeft - 24
    
            waveformString = np.fromfile(fileId, 'int8', 16)
            bytesLeft = bytesLeft - 16
    
            timeTag = np.fromfile(fileId, 'float64', 1)
            bytesLeft = bytesLeft - 8
    
            segmentIndex = np.fromfile(fileId, 'uint32', 1)
            bytesLeft = bytesLeft - 4

            # skip over any remaining data in the header
            np.fromfile(fileId, 'int8', bytesLeft)

            print( "waveformIndex {}, nWaveformBuffers = {}, nPoints = {}, count = {}".format(
                waveformIndex, nWaveformBuffers, nPoints, count)
                 )
            print( "xIncrement {}, xOrigin {}".format( xIncrement, xOrigin ) )
    
            # generate time vector from xIncrement and xOrigin values
            #if (waveformIndex == waveformSelect):
            #    timeVector = xIncrement * np.arange(nPoints) + xOrigin
    
            for bufferIndex in range(nWaveformBuffers):
                # read waveform buffer header
                headerSize = np.fromfile(fileId, 'int32', 1)[0]
                bytesLeft = headerSize - 4
        
                bufferType = np.fromfile(fileId, 'int16', 1)
                bytesLeft = bytesLeft - 2
        
                bytesPerPoint = np.fromfile(fileId, 'int16', 1)[0]
                bytesLeft = bytesLeft - 2
        
                bufferSize = np.fromfile(fileId, 'int32', 1)[0]
                bytesLeft = bytesLeft - 4

                # skip over any remaining data in the header
                np.fromfile(fileId, 'int8', bytesLeft)

                print( "bufferIndex {}, bytesPerPoint {}, bufferSize {}, bufferType {}".format(
                    bufferIndex, bytesPerPoint, bufferSize, bufferType) 
                     )
        
                if (waveformIndex == waveformSelect):
                    if ((bufferType == 1) or (bufferType == 2) or (bufferType == 3)):
                        # bufferType is PB_DATA_NORMAL, PB_DATA_MIN, or PB_DATA_MAX (float)
                        voltageVector = np.fromfile(fileId, 'float32', nPoints)
                    elif (bufferType == 4):
                        # bufferType is PB_DATA_COUNTS (int32)
                        voltageVector = np.fromfile(fileId, '*int32', nPoints)
                    elif (bufferType == 5):
                        # bufferType is PB_DATA_LOGIC (int8)
                        voltageVector = np.fromfile(fileId, '*uint8', nPoints)
                    else:
                        # unrecognized bufferType read as unformated bytes
                        voltageVector = np.fromfile(fileId, '*uint8', bufferSize)
                else:
                    np.fromfile(fileId, 'int8', bufferSize )

        return voltageVector

In [6]:
voltage0 = importAgilentBin( binFilename, 0 )

fileCookie = [65 71], fileVersion = [49 48], fileSize = [16000316], nWaveforms = 2
waveformIndex 0, nWaveformBuffers = 1, nPoints = 2000000, count = [1]
xIncrement [  8.00000000e-10], xOrigin [-0.00074067]
bufferIndex 0, bytesPerPoint 4, bufferSize 8000000, bufferType [1]
waveformIndex 1, nWaveformBuffers = 1, nPoints = 2000000, count = [1]
xIncrement [  8.00000000e-10], xOrigin [-0.00074067]
bufferIndex 0, bytesPerPoint 4, bufferSize 8000000, bufferType [1]


In [7]:
voltage1 = importAgilentBin( binFilename, 1 )

fileCookie = [65 71], fileVersion = [49 48], fileSize = [16000316], nWaveforms = 2
waveformIndex 0, nWaveformBuffers = 1, nPoints = 2000000, count = [1]
xIncrement [  8.00000000e-10], xOrigin [-0.00074067]
bufferIndex 0, bytesPerPoint 4, bufferSize 8000000, bufferType [1]
waveformIndex 1, nWaveformBuffers = 1, nPoints = 2000000, count = [1]
xIncrement [  8.00000000e-10], xOrigin [-0.00074067]
bufferIndex 0, bytesPerPoint 4, bufferSize 8000000, bufferType [1]


In [9]:
xIncrement = 8.00000000e-10
xOrigin = -0.00074067
nPoints = 2000000
timeVector = np.arange( nPoints ) * xIncrement + xOrigin

In [10]:
plt.figure()
plt.plot( timeVector * 1e6, voltage0, label="voltage0" )
plt.plot( timeVector * 1e6, voltage1, label="voltage1" )
plt.grid()
plt.legend()

In [11]:
timeVector.shape

(2000000,)

In [20]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [24]:
smoothed0 = moving_average( voltage0, 50 )

In [25]:
smoothed0.shape

(1999951,)

In [29]:
plt.figure()
plt.plot( moving_average( voltage0, 100 ), label="100" )
plt.legend()
plt.grid()

In [ ]:
plt.figure()
plt.plot( timeVector * 1e6, voltage0, label="voltage0" )
plt.plot( timeVector * 1e6, smoothed0, label="voltage0 smoothed" )
plt.grid()
plt.legend()